# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [1]:
import numpy as np
import ufl

from petsc4py import PETSc
from mpi4py import MPI
from dolfinx import fem, mesh, plot
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD,[[0.0,0.0,0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron)
V = fem.VectorFunctionSpace(domain, ("Lagrange", 2))

INFO:root:running build_ext
INFO:root:building 'libffcx_elements_2d5c9a986741d359a5c76c1d0ce7466e4777b687' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/ffcx/codegeneration -I/usr/include/python3.10 -c libffcx_elements_2d5c9a986741d359a5c76c1d0ce7466e4777b687.c -o ./libffcx_elements_2d5c9a986741d359a5c76c1d0ce7466e4777b687.o -O2
INFO:root:x86_64-linux-gnu-gcc -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 ./libffcx_elements_2d5c9a986741d359a5c76c1d0ce7466e4777b687.o -L/usr/lib/x86_64-linux-gnu -o ./libffcx_elements_2d5c9a986741d359a5c76c1d0ce7466e4777b687.cpython-310-x86_64-linux-gnu.so


We create two python functions for determining the facets to apply boundary conditions to

In [2]:
def left(x):
    return np.isclose(x[0], 0)

def right(x):
    return np.isclose(x[0], L)

fdim = domain.topology.dim -1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

Next, we create a  marker based on these two functions

In [3]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [4]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=PETSc.ScalarType)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [5]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`). 

In [6]:
B = fem.Constant(domain, PETSc.ScalarType((0, 0, 0)))
T = fem.Constant(domain, PETSc.ScalarType((0, 0, 0)))

Define the test and solution functions on the space $V$

In [7]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [8]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J  = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [9]:
# Elasticity parameters
E = PETSc.ScalarType(1.0e4)
nu = PETSc.ScalarType(0.3)
mu = fem.Constant(domain, E/(2*(1 + nu)))
lmbda = fem.Constant(domain, E*nu/((1 + nu)*(1 - 2*nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [10]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [11]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx - ufl.inner(v, T)*ds(2) 

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [12]:
problem = fem.petsc.NonlinearProblem(F, u, bcs)

INFO:root:running build_ext
INFO:root:building 'libffcx_forms_69a876110b554279567433f944d3802b15088492' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/ffcx/codegeneration -I/usr/include/python3.10 -c libffcx_forms_69a876110b554279567433f944d3802b15088492.c -o ./libffcx_forms_69a876110b554279567433f944d3802b15088492.o -O2
INFO:root:x86_64-linux-gnu-gcc -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 ./libffcx_forms_69a876110b554279567433f944d3802b15088492.o -L/usr/lib/x86_64-linux-gnu -o ./libffcx_forms_69a876110b554279567433f944d3802b15088492.cpython-310-x86_64-linux-gnu.so
INFO:root:running build_ext
INFO:root:building 'libffcx_forms_78b407f829a4cb0618fde7e20bfee058f65b12bd' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -

and then create and customize the Newton solver

In [13]:
from dolfinx import nls
solver = nls.petsc.NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"


We create a function to plot the solution at each time step.

In [14]:
import pyvista
import matplotlib.pyplot as plt
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("deformation.gif", fps=3)

topology, cells, geometry = plot.create_vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.FunctionSpace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

INFO:root:running build_ext
INFO:root:building 'libffcx_elements_1779003a382381322b01b435c8795eb07e36a470' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/usr/local/lib/python3.10/dist-packages/ffcx/codegeneration -I/usr/include/python3.10 -c libffcx_elements_1779003a382381322b01b435c8795eb07e36a470.c -o ./libffcx_elements_1779003a382381322b01b435c8795eb07e36a470.o -O2
INFO:root:x86_64-linux-gnu-gcc -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 ./libffcx_elements_1779003a382381322b01b435c8795eb07e36a470.o -L/usr/lib/x86_64-linux-gnu -o ./libffcx_elements_1779003a382381322b01b435c8795eb07e36a470.cpython-310-x86_64-linux-gnu.so
INFO:root:running build_ext
INFO:root:building 'libffcx_expressions_1c636c4fefab7bb38773748e9689319cbb908186' extension
INFO:root:x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DND

Finally, we solve the problem over several time steps, updating the y-component of the traction

In [15]:
from dolfinx import log
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert(converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    plotter.update_coordinates(warped_n.points.copy(), render=False)
    plotter.update_scalar_bar_range([0, 10])
    plotter.update_scalars(magnitude.x.array)
    plotter.write_frame()
plotter.close()

2023-02-01 20:27:09.780 (   7.448s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:10.402 (   8.070s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.622050, 0.560000, 0.060000 (PETSc Krylov solver)
2023-02-01 20:27:10.645 (   8.313s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:11.132 (   8.801s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.487962, 0.450000, 0.050000 (PETSc Krylov solver)
2023-02-01 20:27:11.148 (   8.816s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 22.2455 (tol = 1e-08) r (rel) = 0.134278(tol = 1e-08)
2023-02-01 20:27:11.368 (   9.036s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:11.786 (   9.454s) [main            ]         TimeLogger.cpp:

Time step 1, Number of iterations 8, Load [ 0.   0.  -1.5]


2023-02-01 20:27:15.882 (  13.550s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:16.315 (  13.983s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.433116, 0.410000, 0.030000 (PETSc Krylov solver)
2023-02-01 20:27:16.548 (  14.216s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:17.003 (  14.671s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.454631, 0.400000, 0.050000 (PETSc Krylov solver)
2023-02-01 20:27:17.015 (  14.683s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 17.3254 (tol = 1e-08) r (rel) = 0.117842(tol = 1e-08)
2023-02-01 20:27:17.241 (  14.910s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:17.708 (  15.376s) [main            ]         TimeLogger.cpp:

Time step 2, Number of iterations 9, Load [ 0.  0. -3.]


2023-02-01 20:27:22.203 (  19.871s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:22.672 (  20.341s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.469984, 0.440000, 0.040000 (PETSc Krylov solver)
2023-02-01 20:27:22.937 (  20.605s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:23.397 (  21.065s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.460550, 0.420000, 0.050000 (PETSc Krylov solver)
2023-02-01 20:27:23.411 (  21.080s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 10.0011 (tol = 1e-08) r (rel) = 0.0887471(tol = 1e-08)
2023-02-01 20:27:23.661 (  21.329s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:24.066 (  21.734s) [main            ]         TimeLogger.cpp

...

2023-02-01 20:27:52.656 (  50.324s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:53.116 (  50.784s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.459796, 0.420000, 0.050000 (PETSc Krylov solver)
2023-02-01 20:27:53.403 (  51.071s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:53.875 (  51.543s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.472060, 0.440000, 0.030000 (PETSc Krylov solver)
2023-02-01 20:27:53.889 (  51.557s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 1.06336 (tol = 1e-08) r (rel) = 0.031085(tol = 1e-08)
2023-02-01 20:27:54.139 (  51.807s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:54.614 (  52.282s) [main            ]         TimeLogger.cpp:

Time step 8, Number of iterations 6, Load [  0.   0. -12.]


2023-02-01 20:27:57.015 (  54.683s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:57.430 (  55.098s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.414758, 0.360000, 0.060000 (PETSc Krylov solver)
2023-02-01 20:27:57.667 (  55.335s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:58.071 (  55.739s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.403685, 0.370000, 0.040000 (PETSc Krylov solver)
2023-02-01 20:27:58.083 (  55.751s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 0.898789 (tol = 1e-08) r (rel) = 0.0309666(tol = 1e-08)
2023-02-01 20:27:58.307 (  55.975s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:27:58.760 (  56.428s) [main            ]         TimeLogger.cp

Time step 9, Number of iterations 6, Load [  0.    0.  -13.5]


2023-02-01 20:28:00.342 (  58.010s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 5: r (abs) = 7.87183e-06 (tol = 1e-08) r (rel) = 2.71213e-07(tol = 1e-08)
2023-02-01 20:28:00.592 (  58.260s) [main            ]              petsc.cpp:675   INFO| PETSc Krylov solver starting to solve system.
2023-02-01 20:28:01.038 (  58.707s) [main            ]         TimeLogger.cpp:28    INFO| Elapsed wall, usr, sys time: 0.446748, 0.410000, 0.040000 (PETSc Krylov solver)
2023-02-01 20:28:01.049 (  58.717s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 6: r (abs) = 2.81885e-13 (tol = 1e-08) r (rel) = 9.71198e-15(tol = 1e-08)
2023-02-01 20:28:01.049 (  58.717s) [main            ]       NewtonSolver.cpp:255   INFO| Newton solver finished in 6 iterations and 6 linear solver iterations.


<img src="./deformation.gif" alt="gif" class="bg-primary mb-1" width="800px">